# <center><span style="color:#336699">Introdução à Programação com Dados Geoespaciais em Ambientes de Computação Interativa</span></center>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 150%;">
    Aula 02: Manipulação de Dados Vetoriais em Python</br>
    <span style="font-size: 0.75em;">Parte III - Manipulação de Dados Vetoriais em Python</span>
</div>

<br/>

<div style="text-align: center;font-size: 90%;">
    Gilberto Ribeiro de Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Karine Reis Ferreira<sup><a href="https://orcid.org/0000-0003-2656-5504"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Marcos Adami<sup><a href="https://orcid.org/0000-0003-4247-4477"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Thales Sehn Körting<sup><a href="https://orcid.org/0000-0002-0876-0501"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Divisão de Observação da Terra e Geoinformática, Instituto Nacional de Pesquisas Espaciais (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Última Atualização: 30 de Janeiro de 2025
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
    <b>Resumo.</b> Este Jupyter Notebook apresenta uma visão geral de como realizar a leitura de dados vetoriais com a biblioteca <em>Fiona</em>.
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
    <b>Atenção:</b> Este material é baseado nas notas de aula disponível em <a href="https://prog-geo.github.io">https://prog-geo.github.io</a>.
</div>

# <span style="color:#336699">Introdução</span>
<hr style="border:1px solid #0077b9;">

Nesta seção iremos discutir como realizar a leitura de dados geoespaciais representados na forma de uma **coleção de feições** (**feature collection**). 


Para isso, começaremos construindo um pequeno programa que irá abrir um arquivo de dados vetorial no formato **ESRI Shapefile** contendo os limites das Unidades da Federação do Brasil, referentes ao ano de 2018.


<center><img src="../img/dado-vetorial/uf-2018.png" width="80%" /></center>


<center>Fonte: <a href="https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2018/Brasil/BR/br_unidades_da_federacao.zip"><b>IBGE</b></a>.</center>


A leitura será realizada com ajuda da biblioteca **[Fiona](https://fiona.readthedocs.io/en/latest/)**, que oferece facilidades para leitura e escrita de dados geoespaciais codificados nos mais diversos formatos, principalmente, ESRI Shapefile e GeoJSON. Como será visto, sua API funciona nos mesmos moldes dos protocolos de Entrada e Saída (E/S) da API de arquivos da linguagem Python (Python IO).

# <span style="color:#336699">Importação da biblioteca Fiona</span>
<hr style="border:1px solid #0077b9;">

O comando abaixo importa a biblioteca *Fiona*:

In [ ]:
import fiona

Para verificar a versão da *Fiona* em uso:

In [ ]:
fiona.__version__

# <span style="color:#336699">Leitura de Arquivos ESRI Shapefile</span>
<hr style="border:1px solid #0077b9;">

**Exemplo 1.** Abrindo o conjunto de dados com as unidades da federação para leitura e contabilizando o número de feições contidas nesse arquivo:

In [ ]:
with fiona.open("../dados/shp/br_unidades_da_federacao/BRUFE250GC_SIR.shp", "r") as ufs:
    num_feicoes = len(ufs)
    
    print(f"Número de feições: {num_feicoes}.")

Note que podemos também apontar para a URL do conjunto ESRI Shapefile no formato ZIP:

In [ ]:
with fiona.open("zip+https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2018/Brasil/BR/br_unidades_da_federacao.zip", "r") as ufs:
    num_feicoes = len(ufs)
    
    print(f"Número de feições: {num_feicoes}.")

----

**Exemplo 2.** Atravessar os registros do conjunto de dados e apresentar os atributos alfanuméricos:

In [ ]:
with fiona.open("../dados/shp/br_unidades_da_federacao/BRUFE250GC_SIR.shp", "r") as ufs:
    for uf in ufs:
        print(uf["properties"])

----

**Exemplo 3.** Atravessar os registros do conjunto de dados e computar o centróide dos polígonos das unidades federativas.

In [ ]:
from shapely.geometry import shape

with fiona.open("../dados/shp/br_unidades_da_federacao/BRUFE250GC_SIR.shp", "r") as ufs:
    for uf in ufs:
        geom = shape(uf["geometry"])

        centroide = geom.centroid
        
        nome = uf["properties"]["NM_ESTADO"]
        
        print(f"Nome: {nome}; Localização: {centroide.wkt}")

----

**Exemplo 4.** Faça um programa em Python que leia as feições do conjunto de dados das unidades da federação e escreva na tela o nome da unidade e sua área aproximada em $km^2$.

In [ ]:
import pyproj

from shapely.ops import transform

In [ ]:
epsg_4674 = pyproj.CRS('EPSG:4674')  # SIRGAS 2000
epsg_5880 = pyproj.CRS('EPSG:5880')  # SIRGAS 2000 / Brazil Polyconic

crs_transformer = pyproj.Transformer.from_crs(epsg_4674, epsg_5880, always_xy=True).transform

with fiona.open("../dados/shp/br_unidades_da_federacao/BRUFE250GC_SIR.shp") as ufs:
    for uf in ufs:
        nome = uf["properties"]["NM_ESTADO"]
        geom = shape(uf["geometry"])

        geom_policonica = transform(crs_transformer, geom)
        area = geom_policonica.area / 1e6

        nome_fmt = nome.ljust(20, '.')

        print("{} {:.2f}km²".format(nome_fmt, area))

----

**Exemplo 5:** Apresentar as informações do esquema do conjunto de dados, isto é, sua estrutura e na sequência vamos apresentar os dados:

In [ ]:
import fiona

from shapely.geometry import shape
from shapely.geometry import mapping

with fiona.open("../dados/shp/br_unidades_da_federacao/BRUFE250GC_SIR.shp", "r") as ufs:
    # Número de feições
    num_feicoes = len(ufs)
    print(f"Número de feições: {num_feicoes}\n")

    # Sistema de Referência Espacial
    crs = ufs.crs
    print(f"CRS: {crs}\n")

    # Extensão da coleção de feições
    mbr = ufs.bounds
    print(f"xmin: {mbr[0]}, xmax: {mbr[2]}")
    print(f"ymin: {mbr[1]}, ymax: {mbr[3]}\n")

    # Esquema das feições
    schema = ufs.schema

    # Propriedades alfanuméricas
    propriedades = schema["properties"]

    for a, t in propriedades.items():
        print(f"Atributo: {a}, Tipo: {t}")

    # Propriedade geométrica

    tipo_geometrico = schema["geometry"]
    print(f"\nTipo do atributo geométrico: {tipo_geometrico}\n")

    # Acessando cada uma das Unidades Federativas (feições)
    for uf in ufs:
        # obtendo a geometria associada a feição
        geom = shape(uf["geometry"])

        # computando o centróide da geometria recuperada
        centroide = geom.centroid

        # obtendo o atributo NOME associado à feição
        nome = uf["properties"]["NM_ESTADO"]
        print(f"Nome: {nome}, Localização: {centroide.wkt}")

# <span style="color:#336699">Criando um Mapa Interativo</span>
<hr style="border:1px solid #0077b9;">

O exemplo abaixo mostra como apresentar em um mapa interativo a localização dos centróides das unidades da federação. Iremos utiliza a biblioteca `Fiona` para recuperar os dados e a biblioteca **[ipyleaflet](https://ipyleaflet.readthedocs.io/)** como mapa de suporte. O *ipyleaflet* é uma extensão do pacote *Leaflet.js* para o ambiente do Jupyter, que permite a criação de mapas interativos dentro dos Jupyter Notebooks.

Os seguintes objetos do *ipyleaflet* serão utilizados:
- **Map:** Cria um objeto que representa o mapa de referência.

- **Marker:** Permite criar pontos que podem ser inseridos sobre o mapa de referência.


Além desses objetos, usaremos o objeto `HTML` do pacote *ipywidgets* para exibir textos quando uma localização do mapa for selecionada.


A célula a seguir importa os módulos das bibliotecas que serão utilizadas:

In [ ]:
from ipyleaflet import Map, Marker
from ipywidgets import HTML

from shapely.geometry import shape

Vamos criar um mapa centrado nas coordenadas de latitude `-14.2350` e longitude `-51.9253`, com um nível de zoom que permita visualizar o Brasil:

In [ ]:
map = Map(center=(-14.2350, -51.9253), zoom=3)

Agora, podemos atravessar os registros do conjunto de unidades da federação, computar o centróide e adicionar uma marca no mapa:

In [ ]:
with fiona.open("../dados/shp/br_unidades_da_federacao/BRUFE250GC_SIR.shp", "r") as ufs:
    for uf in ufs:
        geom = shape(uf["geometry"])
        centroide = geom.centroid
        
        nome = uf["properties"]["NM_ESTADO"]

        marker = Marker(location=(centroide.y, centroide.x), draggable=False, title=nome)

        message = HTML()
        message.value = f"<b>{nome}</b>"
        
        marker.popup = message
    
        map.add_layer(marker)

map

**Observação:** Atente-se para o fato de que o código acima não considera transformações entre sistemas de coordenadas.

# <span style="color:#336699">Comentários Gerais do Uso da Fiona</span>
<hr style="border:1px solid #0077b9;">

- Essa biblioteca permite realizar a travessia muito facilmente de arquivos vetoriais.

- Junto a essa biblioteca é distribuída uma ferramenta em linha de comando ou CLI chamada **`fio`**:
```bash
    fio cat ../dados/shp/br_unidades_da_federacao/BRUFE250GC_SIR.shp | fio bounds
```

- Essa biblioteca se apoia em outra biblioteca muito importante no universo de software livre que é a GDAL.

# <span style="color:#336699">Referências Bibliográficas</span>
<hr style="border:1px solid #0077b9;">

- [Documentação da biblioteca Fiona](https://fiona.readthedocs.io/). Disponível online.